In [1]:
# MEJNw2yQ
import pandas as pd
import glob
import os

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## Combine GHCN Data

In [ ]:
files = glob.glob('./ghcn_hourly_data/GHCNh_*.parquet')
print(f"Found {len(files)} files")

columns = [
  "Station_ID",
  # "Station_name",
  "DATE",
  "Latitude",
  "Longitude",
  "Elevation",
  "temperature",
  "wind_speed",
  "relative_humidity",
  # "wet_bulb_temperature",
  # "altimeter",
  # "precipitation"
]

N = len(files)

df_stations = []
for i in range(N):
  if i % 100 == 0:
    print(f"Processing file {i} of {N}")
  df = pd.read_parquet(files[i], engine="pyarrow", columns=columns)

  # Interpolate or aggregate the DATE column to get only hourly data
  df['DATE'] = pd.to_datetime(df['DATE'])
  df.set_index('DATE', inplace=True)
  df_hourly = df.resample('1h').mean(numeric_only=True).reset_index()
  df_hourly["Station_ID"] = df.iloc[0]["Station_ID"]
  df_stations.append(df_hourly)

df_stations = pd.concat(df_stations)
df_stations.to_parquet("./output/ghcn_hourly_combined.parquet")

In [136]:
df_hourly_combined = pd.read_parquet("./output/ghcn_hourly_combined.parquet")
df_hourly_combined[df_hourly_combined["Station_ID"] == "AEI0000OMAL"].to_csv("./output/tmp.csv")

## Data Cleaning

In [ ]:
df = pd.read_parquet("./output/ghcn_hourly_combined.parquet")
df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude', 'Elevation': 'elevation', 'DATE': 'ds'}, inplace=True)
print(f"The initial dataset has {len(df)} rows")

df = df[df["temperature"].notna()]
print(f"After dropping stations with missing temperature values, there are {len(df)} rows")

df["month"] = df["ds"].dt.month
df["hour"] = df["ds"].dt.hour

# Drop any stations with less than 80% complete data (0.80 * 8760 = 7008)
station_counts = df.groupby("Station_ID").size().reset_index(name="count")

min_obs = 8760 * 0.80
df = df[df["Station_ID"].isin(station_counts[station_counts["count"] >= min_obs]["Station_ID"])].copy()

print(f"After dropping stations with less than {min_obs} observations, there are {len(df)} rows remaining")

num_stations = df["Station_ID"].nunique()
print(f"There are {len(df) / num_stations} measurements per station on average")

df.to_parquet("./output/ghcn_hourly_dropped_stations.parquet")

In [ ]:
station_counts = df.groupby("Station_ID").size().reset_index(name="count")
station_counts

## Month-Hour Conversion

In [ ]:
# For each location and month, calculate the average temperature at each hour.
df_cleaned = pd.read_parquet("./output/ghcn_hourly_dropped_stations.parquet")
df_cleaned

nan_count_temperature = df_cleaned["temperature"].isna().sum()
print(f"Found {nan_count_temperature} missing temperature values")

df_avg_temp = df_cleaned.groupby(by=["Station_ID", "month", "hour"]).agg({
  "temperature": "mean",
  "wind_speed": "mean",
  "relative_humidity": "mean",
  "elevation": "mean",
  "latitude": "first",
  "longitude": "first",
}).reset_index()

num_stations = df_avg_temp["Station_ID"].nunique()
print(f"There are {num_stations} unique stations")

obs_per_station = len(df_avg_temp) / num_stations
print(f"There are {obs_per_station} observations per station on average")

# Drop any stations with less than 24*12 = 288 observations.
station_counts = df_avg_temp.groupby("Station_ID").size().reset_index(name="count")
df_avg_temp = df_avg_temp[df_avg_temp["Station_ID"].isin(station_counts[station_counts["count"] == 288]["Station_ID"])]

print(f"After dropping stations with less than 288 observations, there are {df_avg_temp['Station_ID'].nunique()} stations remaining")
station_counts = df_avg_temp.groupby("Station_ID").size().reset_index(name="count")

df_avg_temp.to_parquet("./output/ghcn_avg_temp.parquet")

In [ ]:
# Plot an average day for each month for a sample station
df_avg_temp = pd.read_parquet("./output/ghcn_avg_temp.parquet")
plt.rcParams['font.family'] = 'Helvetica Neue'
# station_id = df_avg_temp["Station_ID"].unique()[1000]
station_id = "SAM00041141"
print(f"Plotting average day for station {station_id}")

df_avg_temp_station = df_avg_temp[df_avg_temp["Station_ID"] == station_id].copy()

# Shift hours based on the longitude
num_hours_offset = int(df_avg_temp_station["longitude"].iloc[0] / 15)
print(f"Shifting hours by {num_hours_offset} hours")
if num_hours_offset < 0:
  num_hours_offset = 24 + num_hours_offset

df_avg_temp_station["hour"] = (df_avg_temp_station["hour"] + num_hours_offset) % 24

df_avg_temp_station = df_avg_temp_station.sort_values(by=["month", "hour"]).reset_index()

for month in df_avg_temp_station["month"].unique():
  df_month = df_avg_temp_station[df_avg_temp_station["month"] == month]
  plt.plot(df_month["hour"], df_month["temperature"], label=f"Month {month}")

plt.legend()
plt.title(f"Average Day for Station {station_id}")
plt.xlabel("Hour")
plt.ylabel("Temperature (°C)")
plt.show()

In [57]:
from typing import Dict
import numpy as np
import pandas as pd
from pydantic import BaseModel

# Constants
AIR_DENSITY = 1.225  # kg/m³
AIR_SPECIFIC_HEAT = 1005  # J/(kg·K)
JOULE_TO_WH = 1/3600  # conversion factor


class Params(BaseModel):
  u_value: float          # W/m²·K
  height: float          # m
  infiltration_rate: float  # air changes per hour
  thermal_mass: float    # J/m²·K
  glazing_transmittance: float  # fraction of solar radiation transmitted


def calculate_solar_flux(latitude, hour, day_of_year):
  """
  Calculate solar flux (W/m²) for a given latitude and hour.
  
  Parameters:
  latitude : float
      Latitude in degrees (-90 to 90)
  hour : float
      Hour of the day (0-24)
  day_of_year : int, optional
      Day of the year (1-365). If None, uses current date.
      
  Returns:
  float : Solar flux in W/m²
  """
  # Convert to radians
  lat_rad = np.radians(latitude)
  
  # Solar constant (W/m²)
  solar_constant = 1361
  
  # Calculate declination angle (δ)
  # Using Cooper's equation
  declination = 23.45 * np.sin(np.radians(360/365 * (day_of_year - 81)))
  declination_rad = np.radians(declination)
  
  # Calculate hour angle (ω)
  # Convert hour to solar hour angle (-180 to +180 degrees)
  hour_angle = (hour - 12) * 15
  hour_angle_rad = np.radians(hour_angle)
  
  # Calculate solar elevation angle (α)
  sin_elevation = (np.sin(lat_rad) * np.sin(declination_rad) + 
                  np.cos(lat_rad) * np.cos(declination_rad) * 
                  np.cos(hour_angle_rad))
  elevation_angle = np.arcsin(sin_elevation)
  
  # If sun is below horizon, return 0
  if sin_elevation <= 0:
      return 0
  
  # Calculate atmospheric mass number
  air_mass = 1 / sin_elevation
  
  # Simple atmospheric transmission model
  # Using Meinel's formula for atmospheric transmission
  transmission = 0.7 ** air_mass
  
  # Calculate actual solar flux considering Earth-Sun distance variation
  # Using Spencer's formula for radius vector
  B = 2 * np.pi * (day_of_year - 1) / 365
  radius_vector = (1.000110 + 0.034221 * np.cos(B) + 0.001280 * np.sin(B) +
                  0.000719 * np.cos(2*B) + 0.000077 * np.sin(2*B))
  
  # Calculate final solar flux
  flux = (solar_constant / (radius_vector ** 2)) * sin_elevation * transmission
  
  return max(0, flux)


def simulate_thermal_flux(_df: pd.DataFrame, params: Params, target_temp_daytime: float, target_temp_nighttime: float) -> Dict:
  """Simulate entire day of greenhouse operation."""
  df = _df.copy()

  if "temperature" not in df.columns:
    raise ValueError("DataFrame must contain a 'temperature' column")
  
  if "hour" not in df.columns:
    raise ValueError("DataFrame must contain a 'hour' column")

  # Calculate the temperature difference between the target temperature and the temperature outdoors.
  is_daytime_mask = (df["hour"] >= 6) & (df["hour"] <= 20)
  df.loc[is_daytime_mask, "delta_t"] = target_temp_daytime - df.loc[is_daytime_mask, "temperature"]
  df.loc[~is_daytime_mask, "delta_t"] = target_temp_nighttime - df.loc[~is_daytime_mask, "temperature"]

  # Positive conduction goes from the inside to the outside.
  df["conduction_w_m2"] = params.u_value * df["delta_t"]

  # Positive infiltration goes from the inside to the outside.
  df["infiltration_w_m2"] = params.height * params.infiltration_rate * AIR_DENSITY * AIR_SPECIFIC_HEAT * df["delta_t"]

  # Positive thermal mass goes from the inside to the outside.
  # df["thermal_mass_w_m2"] = params.thermal_mass * df["delta_t"] / 24 * np.sin(2 * np.pi * df["hour"] / 24) * JOULE_TO_WH

  # Passive solar is always positive, and represents heating.
  # df["passive_solar_w_m2"] = calculate_solar_radiation(df, params.glazing_transmittance)
  df["passive_solar_w_m2"] = df.apply(lambda row: calculate_solar_flux(row["latitude"], row["hour"], row["day_of_year"]), axis=1)

  # This is the flux from the outside to the inside.
  df["total_heat_transfer_w_m2"] = (
    df["passive_solar_w_m2"] -
    df["conduction_w_m2"] - 
    df["infiltration_w_m2"]
  )
  
  return df

In [ ]:
df_avg_temp = pd.read_parquet("./output/ghcn_avg_temp.parquet")
print(f"Input has {len(df_avg_temp)} rows")

# Shift all hours based on the longitude.
df_avg_temp["hour_offset"] = int(df_avg_temp["longitude"].iloc[0] / 15)
df_avg_temp.loc[df_avg_temp["hour_offset"] < 0, "hour_offset"] = 24 + df_avg_temp.loc[df_avg_temp["hour_offset"] < 0, "hour_offset"]
df_avg_temp["hour"] = (df_avg_temp["hour"] + df_avg_temp["hour_offset"]) % 24
df_avg_temp.sort_values(by=["Station_ID", "month", "hour"], inplace=True)
df_avg_temp["day_of_year"] = df_avg_temp["month"] / 12 * 365 + 15

print(f"After shifting, there are {len(df_avg_temp)} rows")

station_ids = df_avg_temp["Station_ID"].unique()
print(f"There are {len(station_ids)} unique stations")

params = Params(
  u_value=4.0,            # W/m²·K
  height=4.0,             # m
  infiltration_rate=0.5,  # air changes per hour
  thermal_mass=100000,    # J/m²·K
  glazing_transmittance=0.8,  # fraction
)
target_temp_daytime = 24
target_temp_nighttime = 18
DAYS_PER_MONTH = 30
SECONDS_PER_HOUR = 3600
W_PER_KW = 1000

print("Simulating thermal flux...")
df_thermal_flux = simulate_thermal_flux(df_avg_temp, params, target_temp_daytime, target_temp_nighttime)
print("Done simulating thermal flux.")

df_thermal_flux["hourly_thermal_energy_kwh_m2"] = df_thermal_flux["total_heat_transfer_w_m2"] / W_PER_KW / SECONDS_PER_HOUR
needs_heating_mask = df_thermal_flux["total_heat_transfer_w_m2"] < 0
needs_cooling_mask = df_thermal_flux["total_heat_transfer_w_m2"] > 0

df_thermal_flux["hourly_cooling_load_kwh_m2"] = df_thermal_flux["hourly_thermal_energy_kwh_m2"].abs() * needs_cooling_mask
df_thermal_flux["hourly_heating_load_kwh_m2"] = df_thermal_flux["hourly_thermal_energy_kwh_m2"].abs() * needs_heating_mask

df_thermal_flux["cooling_degree_hours"] = df_thermal_flux["delta_t"].abs() * (df_thermal_flux["delta_t"] < 0)
df_thermal_flux["heating_degree_hours"] = df_thermal_flux["delta_t"].abs() * (df_thermal_flux["delta_t"] > 0)
df_thermal_flux["adjusted_cooling_degree_hours"] = df_thermal_flux["delta_t"].abs() * needs_cooling_mask
df_thermal_flux["adjusted_heating_degree_hours"] = df_thermal_flux["delta_t"].abs() * needs_heating_mask

df_thermal_flux
print(f"After calculations, there are {len(df_thermal_flux)} rows")
df_thermal_flux.to_parquet("./output/ghcn_month_hour_thermal_flux.parquet")

# Add up the total costs for each month by station.
df_total_thermal_costs = df_thermal_flux.groupby(by=["Station_ID", "month"]).agg({
  "hour_offset": "first",
  "latitude": "first",
  "longitude": "first",
  "relative_humidity": "mean",
  "cooling_degree_hours": "sum",
  "heating_degree_hours": "sum",
  "adjusted_cooling_degree_hours": "sum",
  "adjusted_heating_degree_hours": "sum",
  "hourly_cooling_load_kwh_m2": "sum",
  "hourly_heating_load_kwh_m2": "sum",
}).reset_index()

# Multiply hourly values by 30 days.
df_total_thermal_costs["cooling_load_kwh_m2"] = df_total_thermal_costs["hourly_cooling_load_kwh_m2"] * DAYS_PER_MONTH
df_total_thermal_costs["heating_load_kwh_m2"] = df_total_thermal_costs["hourly_heating_load_kwh_m2"] * DAYS_PER_MONTH
df_total_thermal_costs["cooling_degree_hours"] = df_total_thermal_costs["cooling_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["heating_degree_hours"] = df_total_thermal_costs["heating_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["adjusted_cooling_degree_hours"] = df_total_thermal_costs["adjusted_cooling_degree_hours"] * DAYS_PER_MONTH
df_total_thermal_costs["adjusted_heating_degree_hours"] = df_total_thermal_costs["adjusted_heating_degree_hours"] * DAYS_PER_MONTH

df_total_thermal_costs.drop(columns=["hourly_cooling_load_kwh_m2", "hourly_heating_load_kwh_m2"], inplace=True)
df_total_thermal_costs.to_parquet("./output/ghcn_total_thermal_costs.parquet")

df_total_thermal_costs

In [ ]:
df_avg_temp = pd.read_parquet("./output/ghcn_avg_temp.parquet")

# Find stations iwth the highest average temperature.
avg_temp_by_station = df_avg_temp.groupby("Station_ID")["temperature"].mean().reset_index()
avg_temp_by_station.sort_values(by="temperature", ascending=False, inplace=True)
avg_temp_by_station.head(10)
print(avg_temp_by_station.head(10))
# station_id = "AEI0000OMAL"
station_id = avg_temp_by_station.iloc[0]["Station_ID"]
print(station_id)

df_avg_temp_station = df_avg_temp[df_avg_temp["Station_ID"] == station_id].sort_values(by=["month", "hour"])
df_avg_temp_station.to_csv(f"./output/avg_temp_{station_id}.csv")
print(f"Input has {len(df_avg_temp_station)} rows")

df_plot = pd.read_parquet("./output/ghcn_month_hour_thermal_flux.parquet")
df_plot = df_plot[df_plot["Station_ID"] == station_id].copy()
df_plot.to_csv(f"./output/results_{station_id}.csv")

In [ ]:
def plot_degree_hours(df_plot: pd.DataFrame, degree_hours_column: str, title: str, legend: str):
  # Set matplotlib font:
  plt.rcParams['font.family'] = 'Helvetica Neue'
  fig, ax = plt.subplots(figsize=(12, 5), subplot_kw={'projection': ccrs.PlateCarree()})

  # Add map features
  ax.add_feature(cfeature.COASTLINE)
  ax.add_feature(cfeature.BORDERS, linestyle=':')

  # Set map extent with some padding
  ax.set_extent([
    -180, 180,
    -65, 90
  ])

  df_plot = df_plot[[
    "Station_ID", "latitude", "longitude",
    "heating_degree_hours", "cooling_degree_hours",
    "adjusted_heating_degree_hours", "adjusted_cooling_degree_hours",
    "total_cost_per_m2",
  ]].groupby(by=["Station_ID"]).agg({
    "latitude": "first",
    "longitude": "first",
    "heating_degree_hours": "sum",
    "cooling_degree_hours": "sum",
    "adjusted_heating_degree_hours": "sum",
    "adjusted_cooling_degree_hours": "sum",
    "total_cost_per_m2": "sum",
  }).reset_index()

  # # Remove any station IDs with more than the 95th percentile of heating degree hours.
  # df_plot = df_plot[df_plot["heating_degree_hours"] < df_plot["heating_degree_hours"].quantile(0.95)]
  # # Remove any station IDs with more than the 95th percentile of cooling degree hours.
  # df_plot = df_plot[df_plot["cooling_degree_hours"] < df_plot["cooling_degree_hours"].quantile(0.95)]

  # HEATING DEGREE HOURS
  scatter = ax.scatter(
    df_plot['longitude'],
    df_plot['latitude'],
    c=df_plot[degree_hours_column],
    cmap='RdYlBu_r',
    s=3
  )
  plt.title(f"{title}")
  plt.colorbar(scatter, label=legend)
  fig.show()


kwh_per_mmbtu = 293.07
electricity_price_per_kwh = 0.15
natural_gas_price_per_kwh = 10 / kwh_per_mmbtu
print(f"Electricity price per kWh: ${electricity_price_per_kwh}")
print(f"Natural gas price per kWh: ${natural_gas_price_per_kwh}")
cooling_cop = 8
heating_cop = 0.85
cooling_cost_per_mmbtu = electricity_price_per_kwh / cooling_cop * kwh_per_mmbtu
print(f"Cooling cost per mmbtu: ${cooling_cost_per_mmbtu}")

df_plot = pd.read_parquet("./output/ghcn_total_thermal_costs.parquet")

# Remove any station IDs with more than the 95th percentile of heating degree hours.
df_plot = df_plot[df_plot["heating_degree_hours"] < df_plot["heating_degree_hours"].quantile(0.99)]
# Remove any station IDs with more than the 95th percentile of cooling degree hours.
df_plot = df_plot[df_plot["cooling_degree_hours"] < df_plot["cooling_degree_hours"].quantile(0.99)]

df_plot["heating_cost_per_m2"] = df_plot["heating_load_kwh_m2"] * natural_gas_price_per_kwh / heating_cop
df_plot["cooling_cost_per_m2"] = df_plot["cooling_load_kwh_m2"] * electricity_price_per_kwh / cooling_cop
df_plot["total_cost_per_m2"] = df_plot["heating_cost_per_m2"] + df_plot["cooling_cost_per_m2"]

# plot_degree_hours(df_plot, "heating_degree_hours", "Annual Heating-Degree Hours", "degree-hours (celsius)")
plot_degree_hours(df_plot, "cooling_degree_hours", "Annual Cooling-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "adjusted_heating_degree_hours", "Adjusted Annual Heating-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "adjusted_cooling_degree_hours", "Adjusted Annual Cooling-Degree Hours (Celsius)", "degree-hours (celsius)")
# plot_degree_hours(df_plot, "total_cost_per_m2", "Annual energy cost for climate control ($/m²)", "Energy cost ($/m²)")

df_monthly = df_plot[[
  "Station_ID", "month", "latitude", "longitude",
  "heating_degree_hours", "cooling_degree_hours",
  "heating_load_kwh_m2", "cooling_load_kwh_m2",
  "total_cost_per_m2"
]].copy()

# Round values to reduce file size.
for col in ["heating_degree_hours", "cooling_degree_hours", "heating_load_kwh_m2", "cooling_load_kwh_m2", "total_cost_per_m2"]:
  df_monthly[col] = df_monthly[col].round(2)
df_monthly.to_csv("./output/ghcn_monthly_energy_costs.csv", index=False)

df_annual = df_plot.groupby(by=["Station_ID"]).agg({
  "latitude": "first",
  "longitude": "first",
  "heating_degree_hours": "sum",
  "cooling_degree_hours": "sum",
  "heating_load_kwh_m2": "sum",
  "cooling_load_kwh_m2": "sum",
  "total_cost_per_m2": "sum",
}).reset_index()

# Round values to reduce file size.
for col in ["heating_degree_hours", "cooling_degree_hours", "heating_load_kwh_m2", "cooling_load_kwh_m2", "total_cost_per_m2"]:
  df_annual[col] = df_annual[col].round(2)

df_annual[[
  "Station_ID", "latitude", "longitude",
  "heating_degree_hours", "cooling_degree_hours",
  "heating_load_kwh_m2", "cooling_load_kwh_m2",
  "total_cost_per_m2"
]].to_csv("./output/ghcn_annual_energy_costs_full.csv", index=False)

df_annual[["Station_ID", "latitude", "longitude", "heating_degree_hours", "cooling_degree_hours"]].to_csv("./output/ghcn_annual_degree_hours.csv", index=False)
df_annual[["Station_ID", "latitude", "longitude", "total_cost_per_m2"]].to_csv("./output/ghcn_annual_energy_costs.csv", index=False)

In [ ]:
solar_lcoe_price_per_kwh = 0.04
cooling_cost_per_mmbtu = solar_lcoe_price_per_kwh / cooling_cop * kwh_per_mmbtu
print(f"Cooling cost per mmbtu: ${cooling_cost_per_mmbtu}")

Based on hourly NOAA Global Historical Climatology Network (GHCN) data from 2023. Calculated using the difference between the assumed ideal greenhouse temperature of 23°C and the outdoor temperature.

Based on hourly NOAA Global Historical Climatology Network (GHCN) data from 2023.

Assumes an electricity price of $0.15/kWh (COP=8) and a natural gas price of $10/mmbtu (COP=0.85). Models an ideal greenhouse with a U-value of 4 W/m²·K, height of 4 m, infiltration rate of 0.5 air changes per hour, and glazing transmittance of 0.8. Passive solar heating is based on 1080 W/m² of solar radiation and hourly sun angles. Active heating and cooling is applied to maintain the greenhouse at 23°C.